<a href="https://www.kaggle.com/code/sirwanabdolwahedaula/foxtsage-algorithm-implementation?scriptVersionId=218625878" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### **Foxtsage Algorithm Implementation**

Welcome to the Kaggle notebook demonstrating the implementation of the **Foxtsage** algorithm. This notebook provides both the code and detailed insights into how the hybrid **Foxtsage optimizer** works, as well as a comparison of its performance against the **Adam optimizer**.

---

### **Foxtsage: A Hybrid Algorithm for Dynamic Learning Rate Optimization**

The proposed algorithm, **Foxtsage**, is derived from the **Hybrid FOX-TSA** optimization technique. Foxtsage is specifically designed to dynamically update the learning rate of the **Stochastic Gradient Descent (SGD)** optimizer. By combining the strengths of the **FOX (Fox Optimization)** and **TSA (Tree-Seed Algorithm)**, Foxtsage aims to optimize the training process for various machine learning models, ensuring both efficient convergence and enhanced performance.

---

### **Foxtsage: Combining FOX and TSA for Dynamic Learning Rate Optimization**

1. **Hybrid FOX-TSA: The Core of Foxtsage**  
   - **FOX Algorithm**: Provides global exploration of the search space, minimizing the likelihood of getting stuck in local optima.  
   - **TSA Algorithm**: Focuses on local exploitation, refining potential solutions around promising learning rates.  
   - By combining these strategies, the Hybrid FOX-TSA strikes a balance between exploration and exploitation, offering an effective search for the optimal learning rate.

2. **Dynamic Learning Rate Optimization with Foxtsage**  
   - **Foxtsage** utilizes the Hybrid FOX-TSA approach to iteratively optimize the learning rate (\( \eta \)) of the SGD optimizer.  
   - The process involves:  
     - Initializing a population of learning rates within a specified range (e.g., [0.0001, 0.01]).  
     - Applying FOX-inspired updates for global exploration of potential learning rates.  
     - Refining these candidates using TSA’s local exploitation capabilities.  
   - The best-performing learning rate is selected based on a fitness metric, such as the model’s training loss, to guide the optimization process.

3. **Integration into SGD-Based Models**  
   - The dynamically optimized learning rate is used with SGD during the training of various machine learning models, including **Logistic Regression**, **Multi-Layer Perceptron (MLP)**, and **Convolutional Neural Networks (CNN)**.  
   - This dynamic learning rate adjustment helps the SGD optimizer adapt to changing training conditions, improving convergence speed and model performance.

---

### **Key Features and Benefits of Foxtsage**
- **Dynamic Learning Rate Adjustment**: Optimizes the learning rate throughout the training process, improving SGD's efficiency and performance.  
- **Balanced Optimization**: Combines the global search power of FOX with the local refinement abilities of TSA to enhance learning rate tuning.  
- **Versatile Application**: Can be applied to a variety of machine learning models, including classification tasks with datasets like MNIST and CIFAR-10, as well as engineering design problems.  
- **Enhanced Model Performance**: Improves metrics like accuracy, precision, recall, and F1-score, demonstrating Foxtsage’s robustness across multiple tasks.

---

### **Foxtsage Algorithm Implementation**

In this notebook, each code cell is designed to implement the **Foxtsage algorithm** on a different dataset and machine learning model. The purpose is to demonstrate the versatility and robustness of the Foxtsage optimizer across a variety of tasks.

- **Logistic Regression**: This model is used in one section of the notebook, applied to the **MNIST dataset** for digit classification. The **Foxtsage optimizer** adjusts the learning rate of SGD during the training process, aiming to improve the model's accuracy and convergence.
  
- **Multi-Layer Perceptron (MLP)**: In another section, the **Foxtsage algorithm** is tested on the **CIFAR-10 dataset** using an MLP model. The goal is to optimize the learning rate for better performance on more complex, high-dimensional data.

- **Convolutional Neural Networks (CNN)**: The **CNN model** is used with both **CIFAR-10** and **MNIST** datasets to demonstrate how the **Foxtsage optimizer** can be applied to more complex models in deep learning. By dynamically adjusting the learning rate, **Foxtsage** aims to achieve faster convergence and improved accuracy in image classification tasks.

Each dataset and model implementation is supported by the **Foxtsage optimizer**, ensuring that the learning rate is optimized to provide the best results.

---

### **Citations**
1. Aula, S. A., & Rashid, T. A. (2024). Foxtsage vs. Adam: Revolution or Evolution in Optimization? *arXiv*. [https://doi.org/10.48550/arXiv.2412.17855](https://doi.org/10.48550/arXiv.2412.17855)  
2. Aula, S. A., & Rashid, T. A. (2024). FOX-TSA Hybrid Algorithm: Advancing for Superior Predictive Accuracy in Tourism-Driven Multi-Layer Perceptron Models. *Systems and Soft Computing, Volume 6*. [https://doi.org/10.1016/j.sasc.2024.200178](https://doi.org/10.1016/j.sasc.2024.200178)  
3. Aula, S. A., & Rashid, T. A. (2025). FOX-TSA: Navigating Complex Search Spaces and Superior Performance in Benchmark and Real-World Optimization Problems. *Ain Shams Engineering Journal, Volume 16, Issue 1*. [https://doi.org/10.1016/j.asej.2024.103185](https://doi.org/10.1016/j.asej.2024.103185)

---


### **Implementing Foxtsage for Logistic Regression Model on MNIST Dataset**

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load MNIST Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# Logistic Regression Model
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        return self.linear(x)

# Evaluation Function
def evaluate_model(model, loader, criterion):
    model.eval()
    total_loss = 0
    preds, labels = [], []
    with torch.no_grad():
        for images, labels_batch in loader:
            images, labels_batch = images.to(device), labels_batch.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels_batch)
            total_loss += loss.item()
            _, batch_preds = torch.max(outputs, 1)
            preds.extend(batch_preds.cpu().numpy())
            labels.extend(labels_batch.cpu().numpy())
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")
    f1 = f1_score(labels, preds, average="weighted")
    return total_loss / len(loader), accuracy, precision, recall, f1

# Save Metrics to Excel
def save_metrics_to_excel(metrics, filename, sheet_name):
    pd.DataFrame(metrics).to_excel(filename, sheet_name=sheet_name, index=False)

# Hybrid FOX-TSA Implementation
def run_hybrid_fox_tsa(iterations=50, population_size=30):
    input_dim = 28 * 28
    output_dim = 10
    model = LogisticRegression(input_dim, output_dim).to(device)

    learning_rate_bounds = (0.0001, 0.01)
    population = np.random.uniform(learning_rate_bounds[0], learning_rate_bounds[1], population_size)
    best_lr = population[0]
    best_loss = float("inf")

    criterion = nn.CrossEntropyLoss()
    metrics = {"Iteration": [], "Loss": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}

    for iteration in range(iterations):
        start_time = time.time()
        for i in range(population_size):
            r = np.random.rand()
            if r < 0.5:
                population[i] = best_lr * (1 + np.random.randn() * 0.1)
            else:
                population[i] = best_lr / (1 + np.random.randn() * 0.1)

            # Clip learning rate
            population[i] = np.clip(population[i], learning_rate_bounds[0], learning_rate_bounds[1])
            optimizer = optim.SGD(model.parameters(), lr=population[i])

            # Train for one epoch
            model.train()
            epoch_loss = 0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            if epoch_loss < best_loss:
                best_loss = epoch_loss
                best_lr = population[i]

        # Evaluate the model
        eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1 = evaluate_model(model, test_loader, criterion)
        elapsed_time = time.time() - start_time
        metrics["Iteration"].append(iteration + 1)
        metrics["Loss"].append(eval_loss)
        metrics["Accuracy"].append(eval_accuracy)
        metrics["Precision"].append(eval_precision)
        metrics["Recall"].append(eval_recall)
        metrics["F1-Score"].append(eval_f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Hybrid FOX-TSA Iteration {iteration + 1}/{iterations}: Loss = {eval_loss:.4f}, Accuracy = {eval_accuracy:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Adam Implementation
def run_adam(iterations=50):
    input_dim = 28 * 28
    output_dim = 10
    model = LogisticRegression(input_dim, output_dim).to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    metrics = {"Iteration": [], "Loss": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}

    for epoch in range(iterations):
        start_time = time.time()
        model.train()
        epoch_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # Evaluate the model
        eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1 = evaluate_model(model, test_loader, criterion)
        elapsed_time = time.time() - start_time
        metrics["Iteration"].append(epoch + 1)
        metrics["Loss"].append(eval_loss)
        metrics["Accuracy"].append(eval_accuracy)
        metrics["Precision"].append(eval_precision)
        metrics["Recall"].append(eval_recall)
        metrics["F1-Score"].append(eval_f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Adam Epoch {epoch + 1}/{iterations}: Loss = {eval_loss:.4f}, Accuracy = {eval_accuracy:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Run Experiments
print("\nRunning Hybrid FOX-TSA...")
fox_tsa_metrics = run_hybrid_fox_tsa()
save_metrics_to_excel(fox_tsa_metrics, "MNIST_Logistic_Hybrid_FOX_TSA_Metrics.xlsx", "Hybrid_FOX_TSA")

print("\nRunning Adam...")
adam_metrics = run_adam()
save_metrics_to_excel(adam_metrics, "MNIST_Logistic_Adam_Metrics.xlsx", "Adam")

# Plot Results
plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Loss"], label="Hybrid FOX-TSA Loss")
plt.plot(adam_metrics["Iteration"], adam_metrics["Loss"], label="Adam Loss")
plt.title("MNIST Logistic Regression - Loss Comparison", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Loss", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("MNIST_Logistic_Regression_Loss_Comparison.png", dpi=300)
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Accuracy"], label="Hybrid FOX-TSA Accuracy")
plt.plot(adam_metrics["Iteration"], adam_metrics["Accuracy"], label="Adam Accuracy")
plt.title("MNIST Logistic Regression - Accuracy Comparison", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("MNIST_Logistic_Regression_Accuracy_Comparison.png", dpi=300)
plt.show()


# Implementing Logistic Regression with Foxtsage on IMDB Dataset

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import Workbook
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load IMDB Dataset and Preprocess
def load_imdb_data():
    (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

    word_index = imdb.get_word_index()
    reverse_word_index = {v + 3: k for k, v in word_index.items()}
    reverse_word_index[0] = "<PAD>"
    reverse_word_index[1] = "<START>"
    reverse_word_index[2] = "<UNK>"
    reverse_word_index[3] = "<UNUSED>"

    def decode_review(sequence):
        return " ".join([reverse_word_index.get(i, "<UNK>") for i in sequence])

    X_train_text = [decode_review(review) for review in X_train]
    X_test_text = [decode_review(review) for review in X_test]

    vectorizer = CountVectorizer(max_features=10000, binary=True, stop_words="english")
    X_train_bow = vectorizer.fit_transform(X_train_text).toarray()
    X_test_bow = vectorizer.transform(X_test_text).toarray()

    return train_test_split(X_train_bow, y_train, test_size=0.2, random_state=42), X_test_bow, y_test, X_train_bow.shape[1]

(X_train, X_val, y_train, y_val), X_test, y_test, input_dim = load_imdb_data()

# Convert data to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Logistic Regression Model with Dropout
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.dropout(x)
        return self.linear(x)

# Evaluation Function
def evaluate_model(model, loader, criterion):
    model.eval()
    total_loss = 0
    preds, labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
            _, batch_preds = torch.max(outputs, 1)
            preds.extend(batch_preds.cpu().numpy())
            labels.extend(y_batch.cpu().numpy())
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")
    f1 = f1_score(labels, preds, average="weighted")
    return total_loss / len(loader), accuracy, precision, recall, f1

# Save Metrics to Excel
def save_metrics_to_excel(metrics, filename, sheet_name):
    if not os.path.exists(filename):
        wb = Workbook()
        wb.save(filename)
    with pd.ExcelWriter(filename, mode="a", engine="openpyxl") as writer:
        pd.DataFrame(metrics).to_excel(writer, sheet_name=sheet_name, index=False)

# Hybrid FOX-TSA Implementation
def run_hybrid_fox_tsa(iterations=50, population_size=30):
    output_dim = 2
    model = LogisticRegression(input_dim, output_dim).to(device)

    learning_rate_bounds = (0.0001, 0.01)
    population = np.random.uniform(learning_rate_bounds[0], learning_rate_bounds[1], population_size)
    best_lr = population[0]
    best_loss = float("inf")

    criterion = nn.CrossEntropyLoss()
    metrics = {"Iteration": [], "Loss": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}

    for iteration in range(iterations):
        start_time = time.time()
        for i in range(population_size):
            r = np.random.rand()
            if r < 0.5:
                population[i] = best_lr * (1 + np.random.randn() * 0.1)
            else:
                population[i] = best_lr / (1 + np.random.randn() * 0.1)

            population[i] = np.clip(population[i], learning_rate_bounds[0], learning_rate_bounds[1])
            optimizer = optim.SGD(model.parameters(), lr=population[i])

            model.train()
            epoch_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            if epoch_loss < best_loss:
                best_loss = epoch_loss
                best_lr = population[i]

        eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1 = evaluate_model(model, val_loader, criterion)
        elapsed_time = time.time() - start_time
        metrics["Iteration"].append(iteration + 1)
        metrics["Loss"].append(eval_loss)
        metrics["Accuracy"].append(eval_accuracy)
        metrics["Precision"].append(eval_precision)
        metrics["Recall"].append(eval_recall)
        metrics["F1-Score"].append(eval_f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Hybrid FOX-TSA Iteration {iteration + 1}/{iterations}: Loss = {eval_loss:.4f}, Accuracy = {eval_accuracy:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Adam Implementation
def run_adam(iterations=50):
    output_dim = 2
    model = LogisticRegression(input_dim, output_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    metrics = {"Iteration": [], "Loss": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}

    for epoch in range(iterations):
        start_time = time.time()
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1 = evaluate_model(model, val_loader, criterion)
        elapsed_time = time.time() - start_time
        metrics["Iteration"].append(epoch + 1)
        metrics["Loss"].append(eval_loss)
        metrics["Accuracy"].append(eval_accuracy)
        metrics["Precision"].append(eval_precision)
        metrics["Recall"].append(eval_recall)
        metrics["F1-Score"].append(eval_f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Adam Epoch {epoch + 1}/{iterations}: Loss = {eval_loss:.4f}, Accuracy = {eval_accuracy:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Run Experiments
print("\nRunning Hybrid FOX-TSA...")
fox_tsa_metrics = run_hybrid_fox_tsa()
save_metrics_to_excel(fox_tsa_metrics, "IMDB_Logistic_Hybrid_FOX_TSA_Metrics.xlsx", "Hybrid_FOX_TSA")

print("\nRunning Adam...")
adam_metrics = run_adam()
save_metrics_to_excel(adam_metrics, "IMDB_Logistic_Adam_Metrics.xlsx", "Adam")

# Plot Results
plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Loss"], label="Hybrid FOX-TSA Loss")
plt.plot(adam_metrics["Iteration"], adam_metrics["Loss"], label="Adam Loss")
plt.title("IMDB Logistic Regression - Loss Comparison", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Loss", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("IMDB_Logistic_Regression_Loss_Comparison.png", dpi=300)
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Accuracy"], label="Hybrid FOX-TSA Accuracy")
plt.plot(adam_metrics["Iteration"], adam_metrics["Accuracy"], label="Adam Accuracy")
plt.title("IMDB Logistic Regression - Accuracy Comparison", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("IMDB_Logistic_Regression_Accuracy_Comparison.png", dpi=300)
plt.show()


# Foxtsage Optimization for Multi-Layer Perceptron on MNIST Dataset


In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import Workbook
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load MNIST Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

# Multi-Layer Neural Network with Dropout
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        return self.network(x)

# Evaluation Function
def evaluate_model(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for images, targets in loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            _, batch_preds = torch.max(outputs, 1)
            preds.extend(batch_preds.cpu().numpy())
            labels.extend(targets.cpu().numpy())
    return preds, labels

# Save Metrics to Excel
def save_metrics_to_excel(metrics, filename, sheet_name):
    if not os.path.exists(filename):
        wb = Workbook()
        wb.save(filename)

    with pd.ExcelWriter(filename, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        pd.DataFrame(metrics).to_excel(writer, sheet_name=sheet_name, index=False)

# Adam Optimizer Implementation
def run_adam(model, iterations=50):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    metrics = {"Iteration": [], "Training Loss": [], "Validation Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}
    for epoch in range(iterations):
        start_time = time.time()
        model.train()
        epoch_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        elapsed_time = time.time() - start_time

        preds, labels = evaluate_model(model, test_loader)
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average="weighted")
        recall = recall_score(labels, preds, average="weighted")
        f1 = f1_score(labels, preds, average="weighted")

        metrics["Iteration"].append(epoch + 1)
        metrics["Training Loss"].append(epoch_loss / len(train_loader))
        metrics["Validation Accuracy"].append(accuracy)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["F1-Score"].append(f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Adam Epoch {epoch + 1}/{iterations}: Loss = {epoch_loss:.4f}, Accuracy = {accuracy:.4f}, F1-Score = {f1:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Hybrid FOX-TSA Implementation
def run_hybrid_fox_tsa(model, iterations=50, population_size=30):
    learning_rate_bounds = (0.0001, 0.01)
    population = np.random.uniform(learning_rate_bounds[0], learning_rate_bounds[1], population_size)
    best_lr = population[0]
    best_loss = float("inf")

    criterion = nn.CrossEntropyLoss()
    metrics = {"Iteration": [], "Training Loss": [], "Validation Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}

    for epoch in range(iterations):
        start_time = time.time()
        for i in range(population_size):
            # Dynamic learning rate update
            r = np.random.rand()
            if r < 0.5:
                population[i] = best_lr * (1 + np.random.randn() * 0.1)
            else:
                population[i] = best_lr / (1 + np.random.randn() * 0.1)

            # Clip learning rate to bounds
            population[i] = np.clip(population[i], learning_rate_bounds[0], learning_rate_bounds[1])
            optimizer = optim.SGD(model.parameters(), lr=population[i], weight_decay=1e-4)

            model.train()
            epoch_loss = 0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            # Update best learning rate if loss improves
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                best_lr = population[i]

        elapsed_time = time.time() - start_time

        preds, labels = evaluate_model(model, test_loader)
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average="weighted")
        recall = recall_score(labels, preds, average="weighted")
        f1 = f1_score(labels, preds, average="weighted")

        metrics["Iteration"].append(epoch + 1)
        metrics["Training Loss"].append(best_loss / len(train_loader))
        metrics["Validation Accuracy"].append(accuracy)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["F1-Score"].append(f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Hybrid FOX-TSA Epoch {epoch + 1}/{iterations}: Loss = {best_loss:.4f}, Accuracy = {accuracy:.4f}, F1-Score = {f1:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Run Experiments
input_dim = 28 * 28
hidden_dim = 256
output_dim = 10
iterations = 50  # Set iterations to 50

# Run Hybrid FOX-TSA
mlp_fox_tsa = MLP(input_dim, hidden_dim, output_dim).to(device)
print("\nRunning Hybrid FOX-TSA...")
fox_tsa_metrics = run_hybrid_fox_tsa(mlp_fox_tsa, iterations)
save_metrics_to_excel(fox_tsa_metrics, "Metrics_MLP.xlsx", sheet_name="Hybrid_FOX_TSA_MLP2")

# Run Adam
mlp_adam = MLP(input_dim, hidden_dim, output_dim).to(device)
print("\nRunning Adam...")
adam_metrics = run_adam(mlp_adam, iterations)
save_metrics_to_excel(adam_metrics, "Metrics_MLP.xlsx", sheet_name="Adam_MLP2")

# Plot Results
plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Training Loss"], label="Hybrid FOX-TSA Training Loss")
plt.plot(adam_metrics["Iteration"], adam_metrics["Training Loss"], label="Adam Training Loss")
plt.title("MLP Training Loss on MNIST Dataset", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Training Loss", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("MLP_Training_Loss_Comparison.png", dpi=300)
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Validation Accuracy"], label="Hybrid FOX-TSA Validation Accuracy")
plt.plot(adam_metrics["Iteration"], adam_metrics["Validation Accuracy"], label="Adam Validation Accuracy")
plt.title("MLP Validation Accuracy on MNIST Dataset", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Validation Accuracy", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("MLP_Validation_Accuracy_Comparison.png", dpi=300)
plt.show()


# CNN Training Loss on CIFAR-10 Dataset

In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import Workbook
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

# CNN Architecture: c64-c64-c128-1000
class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000, 10)
        )

    def forward(self, x):
        return self.network(x)

# Evaluation Function
def evaluate_model(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for images, targets in loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            _, batch_preds = torch.max(outputs, 1)
            preds.extend(batch_preds.cpu().numpy())
            labels.extend(targets.cpu().numpy())
    return preds, labels

# Save Metrics to Excel
def save_metrics_to_excel(metrics, filename, sheet_name):
    if not os.path.exists(filename):
        wb = Workbook()
        wb.save(filename)

    with pd.ExcelWriter(filename, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        pd.DataFrame(metrics).to_excel(writer, sheet_name=sheet_name, index=False)

# Adam Optimizer Implementation
def run_adam(model, iterations=50):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    metrics = {"Iteration": [], "Training Loss": [], "Validation Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}
    for epoch in range(iterations):
        start_time = time.time()
        model.train()
        epoch_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        elapsed_time = time.time() - start_time

        preds, labels = evaluate_model(model, test_loader)
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average="weighted")
        recall = recall_score(labels, preds, average="weighted")
        f1 = f1_score(labels, preds, average="weighted")

        metrics["Iteration"].append(epoch + 1)
        metrics["Training Loss"].append(epoch_loss / len(train_loader))
        metrics["Validation Accuracy"].append(accuracy)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["F1-Score"].append(f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Adam Epoch {epoch + 1}/{iterations}: Loss = {epoch_loss:.4f}, Accuracy = {accuracy:.4f}, F1-Score = {f1:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Hybrid FOX-TSA Implementation
def run_hybrid_fox_tsa(model, iterations=50, population_size=30):
    learning_rate_bounds = (0.0001, 0.01)
    population = np.random.uniform(learning_rate_bounds[0], learning_rate_bounds[1], population_size)
    best_lr = population[0]
    best_loss = float("inf")

    criterion = nn.CrossEntropyLoss()
    metrics = {"Iteration": [], "Training Loss": [], "Validation Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}

    for epoch in range(iterations):
        start_time = time.time()
        for i in range(population_size):
            # Dynamic learning rate update
            r = np.random.rand()
            if r < 0.5:
                population[i] = best_lr * (1 + np.random.randn() * 0.1)
            else:
                population[i] = best_lr / (1 + np.random.randn() * 0.1)

            # Clip learning rate to bounds
            population[i] = np.clip(population[i], learning_rate_bounds[0], learning_rate_bounds[1])
            optimizer = optim.SGD(model.parameters(), lr=population[i], weight_decay=1e-4)

            model.train()
            epoch_loss = 0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            # Update best learning rate if loss improves
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                best_lr = population[i]

        elapsed_time = time.time() - start_time

        preds, labels = evaluate_model(model, test_loader)
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average="weighted")
        recall = recall_score(labels, preds, average="weighted")
        f1 = f1_score(labels, preds, average="weighted")

        metrics["Iteration"].append(epoch + 1)
        metrics["Training Loss"].append(best_loss / len(train_loader))
        metrics["Validation Accuracy"].append(accuracy)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["F1-Score"].append(f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Hybrid FOX-TSA Epoch {epoch + 1}/{iterations}: Loss = {best_loss:.4f}, Accuracy = {accuracy:.4f}, F1-Score = {f1:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Run Experiments
iterations = 50  # Updated to 50 iterations

# Run Hybrid FOX-TSA
cnn_fox_tsa = CIFAR10_CNN().to(device)
print("\nRunning Hybrid FOX-TSA...")
fox_tsa_metrics = run_hybrid_fox_tsa(cnn_fox_tsa, iterations)
save_metrics_to_excel(fox_tsa_metrics, "Metrics_CIFAR10.xlsx", sheet_name="Hybrid_FOX_TSA_CNN")

# Run Adam
cnn_adam = CIFAR10_CNN().to(device)
print("\nRunning Adam...")
adam_metrics = run_adam(cnn_adam, iterations)
save_metrics_to_excel(adam_metrics, "Metrics_CIFAR10.xlsx", sheet_name="Adam_CNN")

# Plot Results
plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Training Loss"], label="Hybrid FOX-TSA Training Loss")
plt.plot(adam_metrics["Iteration"], adam_metrics["Training Loss"], label="Adam Training Loss")
plt.title("CNN Training Loss on CIFAR-10 Dataset", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Training Loss", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("CNN_Training_Loss_Comparison_CIFAR10.png", dpi=300)
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Validation Accuracy"], label="Hybrid FOX-TSA Validation Accuracy")
plt.plot(adam_metrics["Iteration"], adam_metrics["Validation Accuracy"], label="Adam Validation Accuracy")
plt.title("CNN Validation Accuracy on CIFAR-10 Dataset", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Validation Accuracy", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("CNN_Validation_Accuracy_Comparison_CIFAR10.png", dpi=300)
plt.show()


# CNN_MNIST_Validation 

In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import Workbook
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load MNIST Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

# Convolutional Neural Network (CNN)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        return self.network(x)

# Evaluation Function
def evaluate_model(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for images, targets in loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            _, batch_preds = torch.max(outputs, 1)
            preds.extend(batch_preds.cpu().numpy())
            labels.extend(targets.cpu().numpy())
    return preds, labels

# Save Metrics to Excel
def save_metrics_to_excel(metrics, filename, sheet_name):
    if not os.path.exists(filename):
        wb = Workbook()
        wb.save(filename)

    with pd.ExcelWriter(filename, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        pd.DataFrame(metrics).to_excel(writer, sheet_name=sheet_name, index=False)

# Adam Optimizer Implementation
def run_adam(model, iterations=50):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    metrics = {"Iteration": [], "Training Loss": [], "Validation Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}
    for epoch in range(iterations):
        start_time = time.time()
        model.train()
        epoch_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        elapsed_time = time.time() - start_time

        preds, labels = evaluate_model(model, test_loader)
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average="weighted")
        recall = recall_score(labels, preds, average="weighted")
        f1 = f1_score(labels, preds, average="weighted")

        metrics["Iteration"].append(epoch + 1)
        metrics["Training Loss"].append(epoch_loss / len(train_loader))
        metrics["Validation Accuracy"].append(accuracy)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["F1-Score"].append(f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Adam Epoch {epoch + 1}/{iterations}: Loss = {epoch_loss:.4f}, Accuracy = {accuracy:.4f}, F1-Score = {f1:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Hybrid FOX-TSA Implementation
def run_hybrid_fox_tsa(model, iterations=50, population_size=30):
    learning_rate_bounds = (0.0001, 0.01)
    population = np.random.uniform(learning_rate_bounds[0], learning_rate_bounds[1], population_size)
    best_lr = population[0]
    best_loss = float("inf")

    criterion = nn.CrossEntropyLoss()
    metrics = {"Iteration": [], "Training Loss": [], "Validation Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Time (s)": []}

    for epoch in range(iterations):
        start_time = time.time()
        for i in range(population_size):
            r = np.random.rand()
            if r < 0.5:
                population[i] = best_lr * (1 + np.random.randn() * 0.1)
            else:
                population[i] = best_lr / (1 + np.random.randn() * 0.1)

            population[i] = np.clip(population[i], learning_rate_bounds[0], learning_rate_bounds[1])
            optimizer = optim.SGD(model.parameters(), lr=population[i], weight_decay=1e-4)

            model.train()
            epoch_loss = 0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            if epoch_loss < best_loss:
                best_loss = epoch_loss
                best_lr = population[i]

        elapsed_time = time.time() - start_time

        preds, labels = evaluate_model(model, test_loader)
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average="weighted")
        recall = recall_score(labels, preds, average="weighted")
        f1 = f1_score(labels, preds, average="weighted")

        metrics["Iteration"].append(epoch + 1)
        metrics["Training Loss"].append(best_loss / len(train_loader))
        metrics["Validation Accuracy"].append(accuracy)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["F1-Score"].append(f1)
        metrics["Time (s)"].append(elapsed_time)

        print(f"Hybrid FOX-TSA Epoch {epoch + 1}/{iterations}: Loss = {best_loss:.4f}, Accuracy = {accuracy:.4f}, F1-Score = {f1:.4f}, Time = {elapsed_time:.2f}s")

    return metrics

# Run Experiments
iterations = 50  # Set to 50 iterations

# Run Hybrid FOX-TSA
cnn_fox_tsa = CNN().to(device)
print("\nRunning Hybrid FOX-TSA...")
fox_tsa_metrics = run_hybrid_fox_tsa(cnn_fox_tsa, iterations)
save_metrics_to_excel(fox_tsa_metrics, "Metrics_CNN.xlsx", sheet_name="Hybrid_FOX_TSA_CNN")

# Run Adam
cnn_adam = CNN().to(device)
print("\nRunning Adam...")
adam_metrics = run_adam(cnn_adam, iterations)
save_metrics_to_excel(adam_metrics, "Metrics_CNN.xlsx", sheet_name="Adam_CNN")

# Plot Results
plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Training Loss"], label="Hybrid FOX-TSA Training Loss")
plt.plot(adam_metrics["Iteration"], adam_metrics["Training Loss"], label="Adam Training Loss")
plt.title("CNN Training Loss on MNIST Dataset", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Training Loss", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("CNN_MNIST_Training_Loss_Comparison.png", dpi=300)
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(fox_tsa_metrics["Iteration"], fox_tsa_metrics["Validation Accuracy"], label="Hybrid FOX-TSA Validation Accuracy")
plt.plot(adam_metrics["Iteration"], adam_metrics["Validation Accuracy"], label="Adam Validation Accuracy")
plt.title("CNN Validation Accuracy on MNIST Dataset", fontsize=16)
plt.xlabel("Iterations", fontsize=14)
plt.ylabel("Validation Accuracy", fontsize=14)
plt.legend()
plt.grid()
plt.savefig("CNN_MNIST_Validation_Accuracy_Comparison.png", dpi=300)
plt.show()
how()
